In [ ]:
!pip install bbaug
!pip install pascal_voc_writer
!pip install -U albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 3.9 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.2.1
    Uninstalling albumentations-1.2.1:
      Successfully uninstalled albumentations-1.2.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import albumentations as A
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from bbaug import policies
import cv2
import os
from pascal_voc_writer import Writer
from xml.dom import minidom
import imgaug as ia
import imgaug.augmenters as iaa
import math
import random
import copy
import glob
imagespath = '/content/drive/MyDrive/255 Project/Augmentation/NON_NORM Image/'
random.seed(7)


def readImage(filename):
    # OpenCV uses BGR channels
    img = cv2.imread(imagespath+filename)
    return img


def getCoordinates(filename):
    allbb = []
    xmldoc = minidom.parse(imagespath+filename)
    itemlist = xmldoc.getElementsByTagName('object')

    size = xmldoc.getElementsByTagName('size')[0]
    width = int((size.getElementsByTagName('width')[0]).firstChild.data)
    height = int((size.getElementsByTagName('height')[0]).firstChild.data)

    for item in itemlist:
        classid = (item.getElementsByTagName('name')[0]).firstChild.data
        xmin = ((item.getElementsByTagName('bndbox')[
            0]).getElementsByTagName('xmin')[0]).firstChild.data
        ymin = ((item.getElementsByTagName('bndbox')[
            0]).getElementsByTagName('ymin')[0]).firstChild.data
        xmax = ((item.getElementsByTagName('bndbox')[
            0]).getElementsByTagName('xmax')[0]).firstChild.data
        ymax = ((item.getElementsByTagName('bndbox')[
            0]).getElementsByTagName('ymax')[0]).firstChild.data

        xmin = int(xmin)
        ymin = int(ymin)
        xmax = int(xmax)
        ymax = int(ymax)

        b = [xmin, ymin, xmax, ymax, classid]
        allbb.append(b)
    return allbb


def start():
    count = 3000
    title = ""
    for filename in sorted(os.listdir(imagespath)):

        if filename.endswith(".jpg") or filename.endswith(".JPG"):
            title, ext = os.path.splitext(os.path.basename(filename))
            image = readImage(filename)
        if filename.endswith(".txt"):
            xmlTitle, txtExt = os.path.splitext(os.path.basename(filename))
            if xmlTitle == title:
                # bboxes = getCoordinates(filename)
                bboxes = readYolo(imagespath+xmlTitle+'.txt')
                for i in range(0, 10):
                    img = copy.deepcopy(image)
                    transform = getTransform(i)
                    try:
                        transformed = transform(image=img, bboxes=bboxes)
                        transformed_image = transformed['image']
                        transformed_bboxes = transformed['bboxes']
                        name = title+str(count)+'.jpg'
                        os.chdir('/content/drive/MyDrive/255 Project/Augmentation/Yolo_Augmentation')
                        cv2.imwrite(name, transformed_image)
                        # print(transformed_bboxes)
                        # writeVoc(transformed_bboxes, count, transformed_image)
                        writeYolo(transformed_bboxes, count, title)
                        count = count+1
                    except:
                        print("bounding box issues")
                        pass

                # bboxes = [[int(float(j)) for j in i] for i in bb]


def writeVoc(bboxes, count, image):
    height, width, channels = image.shape
    imagepath = '/content'+str(count)+'.jpg'
    xmlpath = '/content'+str(count)+'.xml'
    writer = Writer(imagepath, width, height)

    for i in bboxes:
        writer.addObject(i[4], int(i[0]), int(i[1]), int(i[2]), int(i[3]))
    writer.save(xmlpath)


def readYolo(filename):
    coords = []
    with open(filename, 'r') as fname:
        for file1 in fname:
            x = file1.strip().split(' ')
            x.append(x[0])
            x.pop(0)
            x[0] = float(x[0])
            x[1] = float(x[1])
            x[2] = float(x[2])
            x[3] = float(x[3])
            coords.append(x)
    return coords


def writeYolo(coords, count, name):

    with open(name+str(count)+'.txt', "w") as f:
        for x in coords:
            f.write("%s %s %s %s %s \n" % (x[-1], x[0], x[1], x[2], x[3]))


def getTransform(loop):
    transform = []
    if loop == 0:
        transform = A.Compose([
            A.HorizontalFlip(p=1),
        ], bbox_params=A.BboxParams(format='yolo'))
    elif loop == 1:
        transform = A.Compose([
            A.RandomBrightnessContrast(p=1),
        ], bbox_params=A.BboxParams(format='yolo'))
    elif loop == 2:
        transform = A.Compose([
            A.MultiplicativeNoise(multiplier=0.5, p=1),
        ], bbox_params=A.BboxParams(format='yolo'))
    elif loop == 3:
        transform = A.Compose([
            A.RandomGamma(gamma_limit=(80, 120),p=1)
        ], bbox_params=A.BboxParams(format='yolo'))
    elif loop == 4:
        transform = A.Compose([
            A.Blur(blur_limit=(3, 18), p=1)
        ], bbox_params=A.BboxParams(format='yolo'))
    elif loop == 5:
        transform = A.Compose([
            A.JpegCompression(quality_lower=0.5, quality_upper=1.0, p=1)
        ], bbox_params=A.BboxParams(format='yolo'))
    elif loop == 6:
        transform = A.Compose([
            A.BBoxSafeRandomCrop(erosion_rate=0.0, always_apply=True, p=1.0)
        ], bbox_params=A.BboxParams(format='yolo'))
    elif loop == 7:
        transform = A.Compose([
            A.GaussNoise(var_limit=(10.0, 25.0), mean=0, per_channel=True, p=1.0),
        ], bbox_params=A.BboxParams(format='yolo'))  
    elif loop == 8:
        transform = A.Compose([
            A.Sharpen (alpha=(0.2, 0.6), lightness=(0.5, 1.0), p=1.0)
        ], bbox_params=A.BboxParams(format='yolo'))
    elif loop == 9:
        transform = A.Compose([
            A.HorizontalFlip(p=1),
            A.Sharpen (alpha=(0.2, 0.6), lightness=(0.5, 1.0), p=0.5),
            A.RandomBrightnessContrast(p=1.0),
            A.Blur(blur_limit=(3, 12), p=0.5)
        ], bbox_params=A.BboxParams(format='yolo'))    
    return transform

start()

bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues
bounding box issues


In [ ]:
print(A.__version__)

1.3.0
